In [8]:
# PY파일에서는 제거
from IPython.core.display import display, HTML
display(HTML(
"""<style>
div.container { width:100% !important; }
div.CodeMirror {font-family: Consolas; font-size: 16pt;}
div.output { font-size: 16pt; font-weight: bold;}
div.input { font-family: Consolas; font-size: 16pt;}
div.prompt { min-width: 100px;}
</style>
"""))

In [9]:
# !pip3 install tensorflow

In [10]:
import re
import pymysql
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

import warnings
warnings.filterwarnings(action='ignore')

okt = Okt()
tokenizer = Tokenizer()

In [11]:
# 모델 복원
loaded_model = tf.keras.models.load_model('best_model.h5')
# loaded_model.summary()

import pickle
#### 중요!!!!!!! Tokenizer object loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [12]:
max_len = 30
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

DB_HOST = 'localhost'
DB_USER = 'pyuser'
DB_PASSWD = 'pyuser'
DB_NAME = 'pyuser'


In [13]:
conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                   db=DB_NAME, charset='utf8')
curs = conn.cursor()
sql = """
     select count(*)
     from tooncomment_best
     where posneg_grade = 'W'
 """
curs.execute(sql)
table_cnt = curs.fetchall() 
conn.close()

run_cnt = table_cnt[0][0]
print('전체 실행건수: ',run_cnt)

전체 실행건수:  110621


In [15]:
for x in range(0, 10+1, 1):
    
    conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                       db=DB_NAME, charset='utf8')
    curs = conn.cursor()
    sql = """
         select no, strComments	 
         from tooncomment_best
         where posneg_grade = 'W'
         limit 1;
     """
    curs.execute(sql)
    table_data = curs.fetchall() 
    conn.close()

    table_data = [list(table_data[x]) for x in range(len(table_data))]       # tupple을 list형태로 변경
    # print( table_data  )     # list 
    no = table_data[0][0]        # DB에서 한줄을 가져와도 ( (  ,  ) ,     ) 나온다.
    content = table_data[0][1]
    # print(no)
    # print(content)
    
    
    if content is None:   # 가져온 데이터가 Null 일때   
        posneg_rate = 0
        posneg_grade = 'NaN'
    elif len(content) == 0 :   # 이미지들로만 컨텐츠가 구성되어 있을때
        posneg_rate = 0
        posneg_grade = 'Empty'    
    else :    
        content = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', content)
        # print(content)  # 불필요한 글자들 정리
        if len(content) == 0:
            posneg_rate = 0
            posneg_grade = 'Filter'        
        else:
            content = okt.morphs(content, stem=True) # 토큰화
            content = [word for word in content if not word in stopwords] # 불용어 제거
            encoded = tokenizer.texts_to_sequences([content]) # 정수 인코딩
            pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
            score = float(loaded_model.predict(pad_new)) # 예측

            if(score > 0.5):
    #             print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
                posneg_rate = score * 100
                posneg_grade = 'Good'
            else:
    #             print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))
                posneg_rate = (1 - score) * 100
                posneg_grade = 'Bad' 

    # 데이터 업데이트
    conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                       db=DB_NAME, charset='utf8')
    curs = conn.cursor()
    sql = """
         update tooncomment_best
         set posneg_rate={posneg_rate}, posneg_grade='{posneg_grade}'
         where no = {no}
     """
    sql = sql.format(posneg_rate=posneg_rate, posneg_grade=posneg_grade, no=no )
    curs.execute(sql)
    conn.commit()
    conn.close()

#     print('no: ',no,' ','posneg_rate: ',posneg_rate,' ','posneg_grade: ',posneg_grade)

    if x % 1000 == 0 and x != 0:   # 1000 개 단위로 출력
        print('\n\n')
        print(x,'건 진행')
        print('\n\n')

1/1 [==============================] - 0s 13ms/step
